# Sentence Classification Using Machine Learning

To classify sentences into one of several Categories, various Machine Learning and Statistical techniques can be used.  This note focuses on using a Supervised Learning mechanism, developing a model trained on a set of pre-classified sentences.

A particular approach is taken, avoiding the use of counting specific types of words (i.e. "question words") in the features and instead considers Part-Of-Speech patterns in a sentence.  For a full model this could be combined with word-count and other features.

**Notebook Process Flow**
1. Load Data
2. Extract Features
3. Build a Model against the Training Data Set and Validate against Test Set
4. Test the Model against a different Data Set

**Dependencies**  

This Python notebook is dependent on the set of classified sentences in **[sentences.csv](https://github.com/edbullen/NLPBot/releases/download/SupportingFiles/sentences.csv)** (or an equivlent set of data) to run.  In addition to this, another data-set called **[pythonFAQ.csv](https://github.com/edbullen/NLPBot/releases/download/SupportingFiles/pythonFAQ.csv)** is used to test the model.  These need to be downloaded in advance and then the path to these files needs to be set manually in the notebook to correctly reference where they are located.

In order to build a classification model, we need to extract some features, and this is the bulk of effort for the task as detailed in this note.  The Python Sci-Kit Learn library contains a comprehensive pre-packaged machine learning algorithms that can then be used with data-set.

The approach for extracting features is demonstrated in part with in-line code in the notebook, but the full set of functionality is wrapped in a bespoke module **[features.py](https://github.com/edbullen/NLPBot/releases/download/SupportingFiles/features.py)** that needs to be downloaded in advance and then the path to this file needs to be set manually in the notebook to correctly reference where it is located.  The output of running various functions in features.py is saved in the file **[featuresDump.csv](https://github.com/edbullen/NLPBot/releases/download/SupportingFiles/featuresDump.csv)** which also needs to be downloaded in advance and stored in the same location as the other CSV files.

To download these files, either clone the whole GitHub Repo https://github.com/edbullen/NLPBot , or download each file individually:
```
wget https://github.com/edbullen/NLPBot/releases/download/SupportingFiles/sentences.csv
wget https://github.com/edbullen/NLPBot/releases/download/SupportingFiles/pythonFAQ.csv
wget https://github.com/edbullen/NLPBot/releases/download/SupportingFiles/features.py
wget https://github.com/edbullen/NLPBot/releases/download/SupportingFiles/featuresDump.csv
```


## 1. Load Data ##

First, load some pre-classified data from a CSV file called "sentences.csv".


In [ ]:
#load 100 sentences with a classification Q/S/C
import numpy as np
import pandas as pd

CODE_LOC = 'D://SugarSync/git/Python/NLPBot/'   # !! Modify to path to "features.py" folder lcoation
DATA_LOC = 'D://SugarSync/git/Python/NLPBot/analysis/sentences.csv'  # !! Modify this to the CSV data location

sentences = pd.read_csv(filepath_or_buffer = DATA_LOC)   

In [ ]:
sentences.head(10)

In [ ]:
sentences.shape

## 2. Feature Engineering - A Non-Standard, Bespoke Approach ##

Chapter 6 of the NLTK Book has a great deal of background and worked examples for classifying text using machine learning algorithms such as Naive Bayes Classifiers.   A different bespoke approach involving home-grown feature engineering and a scikit-learn Random Forest model is outlined in this note.

The code snippet below is an example of taking a sentence and extracting sets of *POS-tag Triples* from it.  We can use this approach for building up features from a sentence by counting occurances of triple-patterns (or other POS-tag patterns).

In [ ]:
# Extract some patterns of PoS sequences
import nltk
from nltk import word_tokenize

list_of_triple_strings = []  # triple sequence of PoS tags
sentence = "Can a dog see in colour?"

sentenceParsed = word_tokenize(sentence)
pos_tags = nltk.pos_tag(sentenceParsed)
pos = [ i[1] for i in pos_tags ]
print("Words mapped to Part of Speech Tags:",pos_tags)
print("PoS Tags:", pos)

n = len(pos)
for i in range(0,n-3):
    t = "-".join(pos[i:i+3]) # pull out 3 list item from counter, convert to string
    list_of_triple_strings.append(t)
    
print("sequences of triples:", list_of_triple_strings)

### Extracting Features ###
After pre-processing the sentences (using the approach above) we can get a set of triples for Questions, Chat, Statements.  There will be a lot of intersection, but hopefully some clear patterns

### The `features.py` Features Generator ###
This is a custom Python module to extract features from a sentence, written for this ChatBot demo.

`features.py` is located here: https://github.com/edbullen/NLPBot/releases/download/SupportingFiles/features.py
    
Just
```
import features
```
and call 
```
features = features_dict(id,sentence, c)
```

to extract a dictionary of features for the given sentence.  

+ The "id" can be any arbirtary ID value - it just get s passed in and passout as an ID identifier in the resultant dictionary.
+ The "c" value can also be any arbitrary value representing the Class label - the idea is to supply an appropriate label so that the dict that is passed back has all the necessary information in it.

The actual features that are generated and the logic behind how this is done is all hard-coded in features.py (it is not paramaterised - a potential enhancement that could be added)

#### `features.py` POS Triples Extract ####

The features.py module includes a function  
```
get_triples(pos)
```  
which returns a string of the form `"POS-POS-POS"`  where "POS" is a Part-Of-Speech tag.

**Example**

In [ ]:
import sys
sys.path.append(CODE_LOC)  # set search path to code cloned from GitHub
import features            # bespoke "feature engineering" module

sentence = "Can a dog see in colour?"

sentence = features.strip_sentence(sentence)
print(sentence)
pos = features.get_pos(sentence)
triples = features.get_triples(pos)

print(triples)

#### Process for Identifying Candidate Features - Analysis in SQL ####
The objective is to identify candidate PoS sequences that signify a liklihood of a Statement / Question / Chat sentence.

Approach: dump all triples for each sentence with sentence-type label ("S"/"Q"/"C") recorded for each item into a SQL database.

**Count all triples**
```sql
SELECT count(*) FROM triples;  
> 360
```  

**Break-down by label type**
```sql
SELECT count(triple),label 
FROM triples
GROUP by label;  

 count(triple) label
>          37  C
>         145  Q
>         178  S
```   

**Common occuring triple sequences by label type ** 
```sql
SELECT label, triple, occurences
FROM
    (SELECT triples.label label, triples.triple triple, count(triples.triple) occurences
    FROM triples,
        (select triple, count(triple) occurences
         from triples
         group by triple) counts
     WHERE counts.occurences > 2
     AND triples.triple = counts.triple
     GROUP BY triples.triple, triples.label
     ORDER BY 2,1
     ) triples_by_label
WHERE occurences > 1
 ;
```

<table align="left">
<tr><td>Q</td> <td align="left"> <font color="red"> CD-VB-VBN </font></td> <td align="left">5</td></tr>
<tr><td>S</td> <td align="left"> <font color="green">DT-JJ-NN </font></td> <td align="left">3</td></tr>
<tr><td bgcolor="lightgrey">Q</td> <td bgcolor="lightgrey" align="left">DT-NN-NN</td> <td bgcolor="lightgrey" align="left">3</td></tr>
<tr><td bgcolor="lightgrey">S</td> <td bgcolor="lightgrey" align="left">DT-NN-NN</td> <td bgcolor="lightgrey" align="left">3</td></tr>
<tr><td>S</td> <td align="left"> <font color="green"> DT-NN-VBZ </font> </td> <td align="left">3</td></tr>
<tr><td>S</td> <td align="left">DT-NNP-IN</td> <td align="left">2</td></tr>
<tr bgcolor="lightgrey"><td bgcolor="lightgrey">Q</td> <td bgcolor="lightgrey" align="left">DT-NNP-NN</td> <td bgcolor="lightgrey" align="left">4</td></tr>
<tr bgcolor="lightgrey"><td>S</td> <td align="left">DT-NNP-NN</td> <td align="left">5</td></tr>
<tr><td>S</td> <td align="left"><font color="green"> DT-NNP-NNP </font> </td> <td align="left">4</td></tr>
<tr><td>S</td> <td align="left"> <font color="green"> IN-DT-NN </font> </td> <td align="left">3</td></tr>
<tr><td bgcolor="lightgrey">Q</td> <td bgcolor="lightgrey" align="left">IN-DT-NNP</td> <td bgcolor="lightgrey" align="left">4</td></tr>
<tr><td bgcolor="lightgrey">S</td> <td bgcolor="lightgrey" align="left">IN-DT-NNP</td> <td bgcolor="lightgrey" align="left">3</td></tr>
<tr><td>S</td> <td align="left"> <font color="green">IN-NN-NNS </font> </td> <td align="left">3</td></tr>
<tr><td>Q</td> <td align="left"> <font color="red">MD-PRP-VB </font> </td> <td align="left">5</td></tr>
<tr><td>Q</td> <td align="left"> <font color="red">MD-VB-CD </font> </td> <td align="left">4</td></tr>
<tr><td>S</td> <td align="left"> <font color="green">MD-VB-VBN </font> </td> <td align="left">3</td></tr>
<tr><td>Q</td> <td align="left"> <font color="red"> NN-IN-DT </font> </td> <td align="left">3</td></tr>
<tr><td bgcolor="lightgrey">Q</td> <td bgcolor="lightgrey" align="left">NN-NN-IN</td> <td bgcolor="lightgrey" align="left">2</td></tr>
<tr><td bgcolor="lightgrey">S</td> <td bgcolor="lightgrey" align="left">NN-NN-IN</td> <td bgcolor="lightgrey" align="left">3</td></tr>
<tr><td>S</td> <td align="left"> <font color="green"> NNP-IN-NNP </font> </td> <td align="left">3</td></tr>
<tr><td>S</td> <td align="left"> <font color="green"> NNP-NNP-NNP </font> </td> <td align="left">14</td></tr>
<tr><td bgcolor="lightgrey">Q</td> <td bgcolor="lightgrey" align="left">NNP-NNP-VBZ</td> <td bgcolor="lightgrey"  align="left">2</td></tr>
<tr><td bgcolor="lightgrey">S</td> <td bgcolor="lightgrey" align="left">NNP-NNP-VBZ</td> <td bgcolor="lightgrey" align="left">4</td></tr>
<tr><td>S</td> <td align="left"> <font color="green"> NNP-VBZ-DT </font></td> <td align="left">8</td></tr>
<tr><td>S</td> <td align="left"> <font color="green"> NNP-VBZ-NNP </font></td> <td align="left">5</td></tr>
<tr><td>S</td> <td align="left"> <font color="green"> NNS-IN-DT </font></td> <td align="left">3</td></tr>
<tr><td>Q</td> <td align="left"> <font color="red"> PRP-VB-PRP </font></td> <td align="left">3</td></tr>
<tr><td>Q</td> <td align="left"> <font color="red"> PRP-WP-NNP </font></td> <td align="left">3</td></tr>
<tr><td>Q</td> <td align="left"> <font color="red"> VB-CD-VB </font> </td> <td align="left">4</td></tr>
<tr><td>Q</td> <td align="left"> <font color="red"> VB-PRP-WP </font></td> <td align="left">3</td></tr>
<tr><td>S</td> <td align="left"> <font color="green"> VB-VBN-IN </font></td> <td align="left">3</td></tr>
<tr><td>S</td> <td align="left"> <font color="green"> VBZ-DT-JJ </font></td> <td align="left">3</td></tr>
<tr><td>Q</td> <td align="left"> <font color="red"> VBZ-DT-NN </font> </td> <td align="left">7</td></tr>
<tr><td bgcolor="lightgrey">Q</td> <td bgcolor="lightgrey" align="left">VBZ-DT-NNP</td> <td bgcolor="lightgrey" align="left">2</td></tr>
<tr><td bgcolor="lightgrey">S</td> <td bgcolor="lightgrey" align="left">VBZ-DT-NNP</td> <td bgcolor="lightgrey" align="left">5</td></tr>
<tr><td bgcolor="lightgrey">Q</td> <td bgcolor="lightgrey" align="left">VBZ-NNP-NNP</td> <td bgcolor="lightgrey" align="left">3</td></tr>
<tr><td bgcolor="lightgrey">S</td> <td bgcolor="lightgrey" align="left">VBZ-NNP-NNP</td> <td bgcolor="lightgrey" align="left">5</td></tr>
<tr><td>Q</td> <td align="left"> <font color="red"> WP-VBZ-DT </font> </td> <td align="left">6</td></tr>
<tr><td>Q</td> <td align="left"> <font color="red"> WP-VBZ-NNP </font> </td> <td align="left">9</td></tr>
<tr><td>Q</td> <td align="left"> <font color="red"> WRB-MD-VB </font> </td> <td align="left">4</td></tr>
<tr><td>C</td> <td align="left">WRB-VBP-PRP</td> <td align="left">3</td></tr>
</table>



In [ ]:
#### Bespoke Features Generator Example - Get a Python Dictionary of Features ####
sentences = ["Can a dog see in colour?",
             "Hey, How's it going?",
             "Oracle 12.2 will be released for on-premises users on 15 March 2017",
             "When will Oracle 12 be released"]
id = 1
for s in sentences:
    features_dict = features.features_dict(str(id),s)
    features_string,header = features.get_string(str(id),s)
    print(features_dict)
    #print(features_string)
    id += 1

With this approach we can bulk generate some numeric data-features generated from a CSV file of sentences. If each sentence has a unique ID and we have a classifier label (S/Q/C) for each row observation, we can now try to build a ML classification model and assess it's effectiveness.

The script `featuresDump.py` processes a raw `sentences.csv` file with the `features.py` utility and dumps out a file in a format as listed below:

 ```
 id, wordCount, stemmedCount, stemmedEndNN, CD, NN, NNP, NNPS, NNS, PRP, VBG, VBZ, startTuple0, endTuple0, endTuple1, endTuple2, verbBeforeNoun, qMark, qVerbCombo, qTripleScore, sTripleScore, class  
 44d8a78d2ca66b1b, 7, 5, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, S  
 a9133770c79b2c43, 7, 4, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 2, C  
 246cf41a55627762, 5, 3, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, Q  
 53ac5757399632e8, 6, 4, 0, 0, 0, 3, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 2, S  
 78e580bde0b4396e, 6, 4, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, Q  
...  
...  
...  
 036d7e8be25c3108, 4, 2, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, Q  
 b2dd2ca708214c2a, 6, 4, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 4, 0, Q  
 73ebcc1f94f38ddf, 3, 2, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, C  
 617c60a010967c8a, 4, 3, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, C  
 ecef7fa7fcb25f20, 9, 6, 0, 2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, S  
 16fb4f28223d22a9, 7, 5, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, Q  
 7fea2d04212f8039, 8, 5, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, S  
 3df9464caeef89a4, 13, 7, 0, 0, 3, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 3, S  
```


## 3. Build a Machine Learning Model  ##

In this section we load a features CSV file called **`featuresDump.csv`** into a Pandas data-frame.  The data was generated with `features.py` reading in the `sentences.csv` file as described in the previous section.  The featuresDump.csv data is then used to train a Random Forest model to predict whether a sentence is **C**hat, **S**tatement or **Q**uestion.

The `featuresDump.csv` file can be downloaded from here: https://github.com/edbullen/NLPBot/releases/download/SupportingFiles/featuresDump.csv

#### Load the Data ####

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

FNAME = 'D://SugarSync/git/Python/NLPBot/analysis/featuresDump.csv' # !! Modify this to the CSV data location

df = pd.read_csv(filepath_or_buffer = FNAME, )   
print(str(len(df)), "rows loaded")

# Strip any leading spaces from col names
df.columns = df.columns[:].str.strip()
df['class'] = df['class'].map(lambda x: x.strip())

width = df.shape[1]

#### Split into Test and Training Sets ####

In [ ]:
#split into test and training (is_train: True / False col)
np.random.seed(seed=1)
df['is_train'] = np.random.uniform(0, 1, len(df)) <= .75
train, test = df[df['is_train']==True], df[df['is_train']==False]
print(str(len(train)), " rows split into training set,", str(len(test)), "split into test set.")

features = df.columns[1:width-1]  #remove the first ID col and last col=classifier
print("FEATURES = {}".format(features))

#### Fit a Model with the Training Data-Set ####

In [ ]:
# Fit an RF Model for "class" given features
clf = RandomForestClassifier(n_jobs=2, n_estimators = 100)
clf.fit(train[features], train['class'])

#### Generate Predictions from the Test Data-Set ####

In [ ]:


# Predict against test set
preds = clf.predict(test[features])
predout = pd.DataFrame({ 'id' : test['id'], 'predicted' : preds, 'actual' : test['class'] })

In [ ]:
print(predout)

#### Basic Validation ####

In [ ]:
## Cross-check accuracy ##
print(pd.crosstab(test['class'], preds, rownames=['actual'], colnames=['preds']))
print("\n",pd.crosstab(test['class'], preds, rownames=['actual']
                       , colnames=['preds']).apply(lambda r: round(r/r.sum()*100,2), axis=1) )

from sklearn.metrics import accuracy_score
print("\n\nAccuracy Score: ", round(accuracy_score(test['class'], preds),3) ) # https://en.wikipedia.org/wiki/Jaccard_index

#### Flaws in the Approach and Further Validation ####

The accuracy appears pretty good, but the approach taken probably means we have over-fitted the feature selection.  In the next section we try out the model on a completely different data-set, taken from the Python FaQ at https://docs.python.org/3/faq/general.html

## 4. Test Model Against the Python FAQ ##

#### Generate Features using the Features Generator ####
A prepared CSV containing Statements and Questions from the Python FAQ site (https://docs.python.org/3/faq/general.html) can be downloaded from here: https://github.com/edbullen/NLPBot/releases/download/SupportingFiles/pythonFAQ.csv

Some random chat statements have been added to the file as well - EG "What do you reckon?" and "yeah, whatever".

#### Load Sentence Data and Generate Features ####

In [ ]:
# load in some pre-formated FAQ data in a CSV
FNAME = 'D://SugarSync/git/Python/NLPBot/analysis/pythonFAQ.csv' # !! Modify this to the CSV data location

import csv
import hashlib 

import features

fin = open(FNAME, 'rt')
reader = csv.reader(fin)

keys = ["id",
"wordCount",
"stemmedCount",
"stemmedEndNN",
"CD",
"NN",
"NNP",
"NNPS",
"NNS",
"PRP",
"VBG",
"VBZ",
"startTuple0",
"endTuple0",
"endTuple1",
"endTuple2",
"verbBeforeNoun",
"qMark",
"qVerbCombo",
"qTripleScore",
"sTripleScore",
"class"]

rows = []

next(reader)  #Assume we have a header 
for line in reader:
    sentence = line[0]  
    c = line[1]        #class-label
    id = hashlib.md5(str(sentence).encode('utf-8')).hexdigest()[:16] # generate a unique ID
    
    f = features.features_dict(id,sentence, c)
    row = []
    
    for key in keys:
        value = f[key]
        row.append(value)
    rows.append(row)
    
faq = pd.DataFrame(rows, columns=keys)
fin.close()

#### Predict the Class of Sentence with Previously Built Model ####

In [ ]:
# Predict against FAQ test set
featureNames = faq.columns[1:width-1]  #remove the first ID col and last col=classifier
faqPreds = clf.predict(faq[featureNames])

predout = pd.DataFrame({ 'id' : faq['id'], 'predicted' : faqPreds, 'actual' : faq['class'] })

#### Cross-Check Accuracy ####

In [ ]:
## Cross-check accuracy ##
print(pd.crosstab(faq['class'], faqPreds, rownames=['actual'], colnames=['preds']))

print("\n",pd.crosstab(faq['class'], faqPreds, rownames=['actual'],
                       colnames=['preds']).apply(lambda r: round(r/r.sum()*100,2), axis=1) )


In [ ]:
print("Accuracy Score:", round(accuracy_score(faq['class'], faqPreds) ,3) )

This could be summarised as "OK" but not great ...  

The Question and Statement predictions are reported as greater than 80% accurate and the features extraction method could easily be expanded on and enhanced.  
Also the training data-set is small.

## 5. Ad-hoc testing and experiments ##

In [ ]:
textout = {'Q': "QUESTION", 'C': "CHAT", 'S':"STATEMENT"}

mySentence = "Scikit-learn is a popular Python library for Machine Learning."
#mySentence = "The cat is dead"
#mySentence = "Is the cat dead"

myFeatures = features.features_dict('1',mySentence, 'X')

values=[]
for key in keys:
    values.append(myFeatures[key])

s = pd.Series(values)
width = len(s)
myFeatures = s[1:width-1]  #All but the last item (this is the class for supervised learning mode)
predict = clf.predict([myFeatures])

print("\n\nPrediction is: ", textout[predict[0].strip()])